In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [2]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [3]:
def scrape_links(page_number):
    url=f"https://jabar.suara.com/indeks?page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('div',{"class": "text-list-item-y"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 20 links from page 1
Total Links: 20


In [5]:
def scrape_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        
        # Judul Berita
        title_elem = soup.find('h1')
        if title_elem:
            title_text = title_elem.text.strip()
        else:
            title_text = "Title not found"
        # Author berita
        author_elem = soup.find('div', {"class": "writer"})
        if author_elem:
            author_text = author_elem.find('span')
            author_text = author_text.text
        else:
            author_text = "Author not found"     
        # tanggal berita
        date_elem = soup.find('div', {"class": "date-article"})
        if date_elem:
            date_text = date_elem.find('span')
            date_text = date_text.text
        else:
            date_text = "Date not found"
        #     # Category berita
        category_elements = soup.find('div', {"class": "top-detail"})
        if category_elements:
            category_text = category_elements.find('a')
            category_text= category_text.text
        else:
            category_text = "Category not found"
        #     # Content Berita
        body_elem = soup.find('article', {"class": "detail-content detail-berita"})
        
        if body_elem:
            content_elem = body_elem.find_all('p')
            content_text = ""
            for p in content_elem:
                content_text += p.text.strip() + "\n"
            
            if content_text.strip():
                content_text=content_text
            else:
                content_text ="Content not found"
        else:
          content_text ="Content not found"

        results.append({'title': title_text,
                        'author' : author_text,
                        'category':category_text,
                        'date': date_text,
                        'content' : content_text,
                        'region' : 'jabar',
                        'link' : url})
    else:
        print(f"Failed to retrieve data from {url}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [7]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 20


,title,author,category,date,content,region,link
0,"Persib Resmi Rekrut Goran Paulic, Bojan Hodak:...",Galih Prasetyo,\n News ...,"Selasa, 26 September 2023 | 21:13 WIB",SuaraJabar.id - Persib Bandung resmi mendatang...,jakarta,https://jabar.suara.com/read/2023/09/26/211357...
1,Oknum Polisi di Lembang Tercyduk Merokok Sambi...,Amelia Prisilia,\n News ...,"Selasa, 26 September 2023 | 15:18 WIB",SuaraJabar.id - Belum lama ini netizen ramai m...,jakarta,https://jabar.suara.com/read/2023/09/26/151852...
2,Kronologis Bule Amerika Tega Bunuh Mertua di B...,Galih Prasetyo,\n News ...,"Senin, 25 September 2023 | 17:07 WIB",SuaraJabar.id - Seorang warga negara asing (WN...,jakarta,https://jabar.suara.com/read/2023/09/25/170713...
3,"Daftar ATM BRI Terdekat Pondok Gede, Lengkap d...",Rezza Dwi Rachmanta,\n,"Selasa, 26 September 2023 | 14:04 WIB",SuaraJabar.id - BRI mempunyai lebih dari 15 AT...,jakarta,https://jabar.suara.com/read/2023/09/26/140440...
4,Jadi Korban Begal di Bandung Hingga Motor Raib...,Amelia Prisilia,\n News ...,"Selasa, 26 September 2023 | 13:29 WIB",SuaraJabar.id - Seorang pria membagikan kisahn...,jakarta,https://jabar.suara.com/read/2023/09/26/132952...
5,Lagi! Persib Datangkan Staf Pelatih Asal Kroas...,Amaludin,\n Jabar ...,"Selasa, 26 September 2023 | 14:25 WIB","PURWASUKA - Setelah Miro Petric, Persib kembal...",jakarta,https://purwasuka.suara.com/read/2023/09/26/14...
6,"Prakiraan Cuaca Jawa Barat Hari Ini, 27 Septem...",Amaludin,\n Jabar ...,"Rabu, 27 September 2023 | 08:00 WIB","PURWASUKA - Badan Meteorologi, Klimatologi dan...",jakarta,https://purwasuka.suara.com/read/2023/09/27/08...
7,"Tiket Persib vs Persita Mulai Dijual Hari Ini,...",Amaludin,\n Jabar ...,"Senin, 25 September 2023 | 14:45 WIB",PURWASUKA - Menjelang pekan ke-14 Liga 1 2023/...,jakarta,https://purwasuka.suara.com/read/2023/09/25/14...
8,Produsen Gamis di Soreang Gembira Tiktok Shop ...,Galih Prasetyo,\n News ...,"Selasa, 26 September 2023 | 20:25 WIB",SuaraJabar.id - Produsen pakaian di Kabupaten ...,jakarta,https://jabar.suara.com/read/2023/09/26/202535...
9,Pengasuh Yayasan di Tasikmalaya Diduga Sodomi ...,Galih Prasetyo,\n News ...,"Senin, 25 September 2023 | 17:31 WIB",SuaraJabar.id - Seorang pengasuh yayasan di Ta...,jakarta,https://jabar.suara.com/read/2023/09/25/173105...


In [ ]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/jabarsuara_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')